In [1]:
import sys
sys.path.append('..')
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from collections import Counter

# Load the Data

In [2]:
# Load the data from the Excel file
df = pd.read_excel('players.xlsx')

# Extract player data
players = []
for _, row in df.iterrows():
    players.append((row['Name'], row['Position'], row['Skill'], row['Salary (â‚¬M)']))

In [3]:
df = pd.DataFrame(players, columns=['Player Name', 'Position', 'Rating', 'Performance'])
print(df)

         Player Name Position  Rating  Performance
0        Alex Carter       GK      85           90
1       Jordan Smith       GK      88          100
2      Ryan Mitchell       GK      83           85
3     Chris Thompson       GK      80           80
4    Blake Henderson       GK      87           95
5      Daniel Foster      DEF      90          110
6      Lucas Bennett      DEF      85           90
7        Owen Parker      DEF      88          100
8       Ethan Howard      DEF      80           70
9         Mason Reed      DEF      82           75
10      Logan Brooks      DEF      86           95
11      Caleb Fisher      DEF      84           85
12     Nathan Wright      MID      92          120
13      Connor Hayes      MID      89          105
14      Dylan Morgan      MID      91          115
15     Hunter Cooper      MID      83           85
16     Austin Torres      MID      82           80
17  Gavin Richardson      MID      87           95
18      Spencer Ward      MID  

In [4]:
players

[('Alex Carter', 'GK', 85, 90),
 ('Jordan Smith', 'GK', 88, 100),
 ('Ryan Mitchell', 'GK', 83, 85),
 ('Chris Thompson', 'GK', 80, 80),
 ('Blake Henderson', 'GK', 87, 95),
 ('Daniel Foster', 'DEF', 90, 110),
 ('Lucas Bennett', 'DEF', 85, 90),
 ('Owen Parker', 'DEF', 88, 100),
 ('Ethan Howard', 'DEF', 80, 70),
 ('Mason Reed', 'DEF', 82, 75),
 ('Logan Brooks', 'DEF', 86, 95),
 ('Caleb Fisher', 'DEF', 84, 85),
 ('Nathan Wright', 'MID', 92, 120),
 ('Connor Hayes', 'MID', 89, 105),
 ('Dylan Morgan', 'MID', 91, 115),
 ('Hunter Cooper', 'MID', 83, 85),
 ('Austin Torres', 'MID', 82, 80),
 ('Gavin Richardson', 'MID', 87, 95),
 ('Spencer Ward', 'MID', 84, 85),
 ('Sebastian Perry', 'FWD', 95, 150),
 ('Xavier Bryant', 'FWD', 90, 120),
 ('Elijah Sanders', 'FWD', 93, 140),
 ('Adrian Collins', 'FWD', 85, 90),
 ('Tyler Jenkins', 'FWD', 80, 70),
 ('Chase Murphy', 'FWD', 86, 95),
 ('Landon Powell', 'FWD', 89, 110),
 ('Julian Scott', 'FWD', 92, 130),
 ('Bentley Rivera', 'MID', 88, 100),
 ('Maxwell Flores'

In [18]:
# Constants
NUM_TEAMS = 5
PLAYERS_PER_TEAM = 7
MAX_BUDGET = 750  # Million €
POSITION_REQUIREMENTS = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}

In [19]:
class TeamAssignmentSolution:
    def __init__(self, players, population=None):
        self.players = players  # List of players that will be assigned to teams
        self.population = population if population else {}  # Dictionary with team assignments
    
    def random_initial_representation(self):
        """Randomly create a valid team assignment representation."""
        # Initialize empty teams with required position slots and cost trackers
        teams = {i: {"GK": [], "DEF": [], "MID": [], "FWD": [], "total_cost": 0} for i in range(NUM_TEAMS)}
        
        # Shuffle players to randomly assign them to teams
        shuffled_players = random.sample(self.players, len(self.players))
        
        for player in shuffled_players:
            player_name, position, skill, cost = player
            assigned = False
            # Try to assign the player to a team while respecting position and budget constraints
            for team_id in teams:
                if len(teams[team_id][position]) < POSITION_REQUIREMENTS[position] and teams[team_id]["total_cost"] + cost <= MAX_BUDGET:
                    teams[team_id][position].append(player)
                    teams[team_id]["total_cost"] += cost
                    assigned = True
                    break
            if not assigned:
                continue  # Skip players that can't be assigned under budget or position constraints
        
        # Store the generated team assignment
        self.population = teams
        return self.population
    
    def fitness(self):
        """Evaluate the quality of the solution."""
        team_avg_skills = []
        
        # Iterate over each team in the population
        for team in self.population.values():
            total_skill = 0
            total_players = 0
            
            # Iterate over each position (GK, DEF, MID, FWD)
            for position in ["GK", "DEF", "MID", "FWD"]:
                if len(team[position]) > 0:  # Ensure the position is filled
                    for player in team[position]:
                        total_skill += player[2]  # Add the skill rating of the player
                        total_players += 1  # Count the player
            
            # Calculate the average skill rating for the team
            avg_skill = total_skill / total_players if total_players > 0 else 0
            team_avg_skills.append(avg_skill)

        # Calculate the standard deviation of the average skill ratings across teams
        std_dev = np.std(team_avg_skills)
        return std_dev


In [23]:
# Create an instance of the TeamAssignmentSolution class
solution = TeamAssignmentSolution(players)
solution.random_initial_representation()

# Evaluate the fitness of the initial population
fitness_score = solution.fitness()
print(f"Fitness Score: {fitness_score}")

Fitness Score: 1.4410880243335396


# Selection Mechanism

In Ranking Selection, solutions with better fitness (lower fitness in a minimization problem) should be selected more frequently, as their ranks will be higher.

In [8]:
def ranking_selection(population: list, maximization: bool = False):
    """Ranking Selection"""
    
    # Sort the population based on fitness
    sorted_population = sorted(population, key=lambda ind: ind.fitness())
    
    # For minimization, rank from best to worst (lower fitness gets a higher rank)
    if not maximization:
        sorted_population.reverse()  # Reverse the sorting for minimization (lower fitness first)
    
    # Assign ranks to each individual (1 for the best, 2 for second-best, etc.)
    ranks = list(range(1, len(population) + 1))
    
    # Calculate selection probabilities based on ranks
    total_rank = sum(ranks)
    probabilities = [rank / total_rank for rank in ranks]
    
    # Select an individual based on the rank probabilities
    random_nr = random.uniform(0, 1)
    cumulative_prob = 0
    
    for idx, prob in enumerate(probabilities):
        cumulative_prob += prob
        if random_nr <= cumulative_prob:
            return deepcopy(sorted_population[idx])  # Return the selected individual

In Tournament Selection, since the individuals are randomly chosen, the frequency distribution will depend on the tournament size and randomness of the selections.

In [9]:
def tournament_selection(population: list, tournament_size: int = 3, maximization: bool = False):
    """Tournament Selection"""
    
    # Randomly select a subset of individuals (tournament size)
    tournament_individuals = random.sample(population, tournament_size)
    
    # Sort the tournament individuals by fitness (minimization, so lower is better)
    if maximization:
        best_individual = min(tournament_individuals, key=lambda ind: ind.fitness())  # For maximization: select best
    else:
        best_individual = max(tournament_individuals, key=lambda ind: ind.fitness())  # For minimization: select best
    
    return deepcopy(best_individual)  # Return the best individual from the tournament

In [10]:
def test_selection():
    # Create a small population of TeamAssignmentSolution instances
    population = [TeamAssignmentSolution(players) for _ in range(10)]
    
    # Ensure that each individual has been initialized properly
    for i, sol in enumerate(population):
        sol.random_initial_representation()
        print(f"Solution {i+1} - Fitness: {sol.fitness()}")
    
    # Run the selection process multiple times (e.g., 100 times) for both methods
    ranking_selected_solutions = [ranking_selection(population) for _ in range(100)]
    tournament_selected_solutions = [tournament_selection(population) for _ in range(100)]
    
    # Count how many times each fitness was selected
    ranking_fitness_counter = Counter([sol.fitness() for sol in ranking_selected_solutions])
    tournament_fitness_counter = Counter([sol.fitness() for sol in tournament_selected_solutions])
    
    # Print the selection frequency for both methods
    print("\nRanking Selection Frequency (100 Selections):")
    for fitness, count in ranking_fitness_counter.items():
        print(f"Fitness {fitness}: Selected {count} times")
    
    print("\nTournament Selection Frequency (100 Selections):")
    for fitness, count in tournament_fitness_counter.items():
        print(f"Fitness {fitness}: Selected {count} times")

# Run the test
test_selection()

Solution 1 - Fitness: 1.2853967861945135
Solution 2 - Fitness: 1.2067835475395925
Solution 3 - Fitness: 2.495347144109314
Solution 4 - Fitness: 1.386229653795222
Solution 5 - Fitness: 1.0552105892925367
Solution 6 - Fitness: 1.2202375145178639
Solution 7 - Fitness: 0.825115947325352
Solution 8 - Fitness: 1.126308762538688
Solution 9 - Fitness: 2.2882129200201042
Solution 10 - Fitness: 1.8547236990991414

Ranking Selection Frequency (100 Selections):
Fitness 0.825115947325352: Selected 23 times
Fitness 1.2202375145178639: Selected 9 times
Fitness 2.495347144109314: Selected 4 times
Fitness 1.126308762538688: Selected 17 times
Fitness 1.0552105892925367: Selected 14 times
Fitness 1.2067835475395925: Selected 9 times
Fitness 1.2853967861945135: Selected 10 times
Fitness 2.2882129200201042: Selected 4 times
Fitness 1.8547236990991414: Selected 5 times
Fitness 1.386229653795222: Selected 5 times

Tournament Selection Frequency (100 Selections):
Fitness 2.495347144109314: Selected 23 times
F

# Mutation Operators

This function performs a verbose mutation by swapping one player of the same position between two randomly chosen teams and printing both teams’ lineups before and after the swap for clarity.

In [ ]:
def mutate_swap_same_position(solution: TeamAssignmentSolution):
    team_ids = list(solution.population.keys())
    t1, t2 = random.sample(team_ids, 2)
    pos = random.choice(["GK", "DEF", "MID", "FWD"])

    if not solution.population[t1][pos] or not solution.population[t2][pos]:
        print("No mutation occurred — one of the teams had no players in position:", pos)
        return

    p1_idx = random.randint(0, len(solution.population[t1][pos]) - 1)
    p2_idx = random.randint(0, len(solution.population[t2][pos]) - 1)

    print(f"\n Swapping between Team {t1} and Team {t2} at position {pos.upper()}")

    print(f"\nBefore Mutation:")
    print(f"Team {t1} {pos}: {[p[0] for p in solution.population[t1][pos]]}")
    print(f"Team {t2} {pos}: {[p[0] for p in solution.population[t2][pos]]}")

    # Swap
    solution.population[t1][pos][p1_idx], solution.population[t2][pos][p2_idx] = \
        solution.population[t2][pos][p2_idx], solution.population[t1][pos][p1_idx]

    print(f"\nAfter Mutation:")
    print(f"Team {t1} {pos}: {[p[0] for p in solution.population[t1][pos]]}")
    print(f"Team {t2} {pos}: {[p[0] for p in solution.population[t2][pos]]}")

# Run the test:
mutate_swap_same_position(solution)



 Swapping between Team 3 and Team 2 at position MID

Before Mutation:
Team 3 MID: ['Nathan Wright', 'Dylan Morgan']
Team 2 MID: ['Austin Torres', 'Connor Hayes']

After Mutation:
Team 3 MID: ['Nathan Wright', 'Austin Torres']
Team 2 MID: ['Dylan Morgan', 'Connor Hayes']


This mutation function swaps two players between different positions within a team, ensuring that the team’s total cost does not exceed the budget limit after the swap.

In [54]:
def mutate_swap_positions_within_team(solution: TeamAssignmentSolution):
    team_ids = list(solution.population.keys())
    team_id = random.choice(team_ids)
    team = solution.population[team_id]

    pos1, pos2 = random.sample(["GK", "DEF", "MID", "FWD"], 2)

    if not team[pos1] or not team[pos2]:
        print("Mutation skipped: one of the positions is empty.")
        return

    idx1 = random.randint(0, len(team[pos1]) - 1)
    idx2 = random.randint(0, len(team[pos2]) - 1)

    p1 = team[pos1][idx1]
    p2 = team[pos2][idx2]

    new_cost = team["total_cost"] - p1[3] - p2[3] + p2[3] + p1[3]

    print(f"\n Swapping within Team {team_id}: {p1[0]} ({pos1}) <-> {p2[0]} ({pos2})")

    print(f"\nBefore Mutation - Team {team_id}:")
    for pos in ["GK", "DEF", "MID", "FWD"]:
        print(f"{pos}: {[p[0] for p in team[pos]]}")
    print("Cost:", team["total_cost"])

    if new_cost <= MAX_BUDGET:
        team[pos1][idx1], team[pos2][idx2] = p2, p1
        team["total_cost"] = new_cost

        print(f"\nAfter Mutation - Team {team_id}:")
        for pos in ["GK", "DEF", "MID", "FWD"]:
            print(f"{pos}: {[p[0] for p in team[pos]]}")
        print("Cost:", team["total_cost"])
    else:
        print("Mutation cancelled: would exceed budget.")

# Run the test:
mutate_swap_positions_within_team(solution)



 Swapping within Team 2: Connor Hayes (MID) <-> Tyler Jenkins (FWD)

Before Mutation - Team 2:
GK: ['Blake Henderson']
DEF: ['Owen Parker', 'Lucas Bennett']
MID: ['Dylan Morgan', 'Connor Hayes']
FWD: ['Tyler Jenkins', 'Ryan Mitchell']
Cost: 725

After Mutation - Team 2:
GK: ['Blake Henderson']
DEF: ['Owen Parker', 'Lucas Bennett']
MID: ['Dylan Morgan', 'Tyler Jenkins']
FWD: ['Connor Hayes', 'Ryan Mitchell']
Cost: 725


This function swaps three players from different positions (GK, DEF, MID, FWD) within a team, ensuring that the team structure (1 GK, 2 DEF, 2 MID, 2 FWD) remains valid, and shuffling the players while preserving the overall team balance.

In [68]:
def mutate_swap_three_players(solution: TeamAssignmentSolution):
    team_to_mutate = random.choice(list(solution.population.keys()))
    positions = ["GK", "DEF", "MID", "FWD"]
    
    # Randomly select three distinct positions to swap players
    pos1, pos2, pos3 = random.sample(positions, 3)
    
    # Ensure all selected positions have players to swap
    if not solution.population[team_to_mutate][pos1] or \
       not solution.population[team_to_mutate][pos2] or \
       not solution.population[team_to_mutate][pos3]:
        return
    
    # Select random players from each position
    player1_idx = random.randint(0, len(solution.population[team_to_mutate][pos1]) - 1)
    player2_idx = random.randint(0, len(solution.population[team_to_mutate][pos2]) - 1)
    player3_idx = random.randint(0, len(solution.population[team_to_mutate][pos3]) - 1)
    
    player1 = solution.population[team_to_mutate][pos1][player1_idx]
    player2 = solution.population[team_to_mutate][pos2][player2_idx]
    player3 = solution.population[team_to_mutate][pos3][player3_idx]
    
    # Print before the mutation
    print(f"\nBefore Mutation:")
    print(f"Team {team_to_mutate} {pos1}: {[p[0] for p in solution.population[team_to_mutate][pos1]]}")
    print(f"Team {team_to_mutate} {pos2}: {[p[0] for p in solution.population[team_to_mutate][pos2]]}")
    print(f"Team {team_to_mutate} {pos3}: {[p[0] for p in solution.population[team_to_mutate][pos3]]}")
    
    # Swap the three players
    solution.population[team_to_mutate][pos1][player1_idx], solution.population[team_to_mutate][pos2][player2_idx], solution.population[team_to_mutate][pos3][player3_idx] = \
        solution.population[team_to_mutate][pos2][player2_idx], solution.population[team_to_mutate][pos3][player3_idx], solution.population[team_to_mutate][pos1][player1_idx]
    
    # Print after the mutation
    print(f"\nAfter Mutation:")
    print(f"Team {team_to_mutate} {pos1}: {[p[0] for p in solution.population[team_to_mutate][pos1]]}")
    print(f"Team {team_to_mutate} {pos2}: {[p[0] for p in solution.population[team_to_mutate][pos2]]}")
    print(f"Team {team_to_mutate} {pos3}: {[p[0] for p in solution.population[team_to_mutate][pos3]]}")

# Run the test
mutate_swap_three_players(solution)



Before Mutation:
Team 4 GK: ['Bentley Rivera']
Team 4 DEF: ['Ethan Howard', 'Jaxon Griffin']
Team 4 MID: ['Xavier Bryant', 'Jordan Smith']

After Mutation:
Team 4 GK: ['Ethan Howard']
Team 4 DEF: ['Jordan Smith', 'Jaxon Griffin']
Team 4 MID: ['Xavier Bryant', 'Bentley Rivera']


# Crossover Operators

In [ ]:
def crossover_team_blocks(parent1: TeamAssignmentSolution, parent2: TeamAssignmentSolution):
    crossover_point = random.randint(1, NUM_TEAMS - 1)
    child1_teams = {}
    child2_teams = {}
    
    for i in range(NUM_TEAMS):
        if i < crossover_point:
            child1_teams[i] = deepcopy(parent1.population[i])
            child2_teams[i] = deepcopy(parent2.population[i])
        else:
            child1_teams[i] = deepcopy(parent2.population[i])
            child2_teams[i] = deepcopy(parent1.population[i])
    
    return TeamAssignmentSolution(parent1.players, child1_teams), TeamAssignmentSolution(parent2.players, child2_teams)

In [106]:
parent1 = TeamAssignmentSolution(players)
parent1.random_initial_representation()

parent2 = TeamAssignmentSolution(players)
parent2.random_initial_representation()

# Perform crossover
offspring1, offspring2 = crossover_team_blocks(parent1, parent2)

# Print results for offspring1 and offspring2
print("Offspring 1:")
for team_id, team in offspring1.population.items():
    print(f"Team {team_id}: {team}")

print("\nOffspring 2:")
for team_id, team in offspring2.population.items():
    print(f"Team {team_id}: {team}")

Offspring 1:
Team 0: {'GK': [('Blake Henderson', 'GK', 87, 95)], 'DEF': [('Logan Brooks', 'DEF', 86, 95), ('Brayden Hughes', 'DEF', 87, 100)], 'MID': [('Ashton Phillips', 'MID', 90, 110), ('Dominic Bell', 'MID', 86, 95)], 'FWD': [('Zachary Nelson', 'FWD', 86, 92), ('Colton Gray', 'FWD', 91, 125)], 'total_cost': 712}
Team 1: {'GK': [('Alex Carter', 'GK', 85, 90)], 'DEF': [('Daniel Foster', 'DEF', 90, 110), ('Maxwell Flores', 'DEF', 81, 72)], 'MID': [('Bentley Rivera', 'MID', 88, 100), ('Dylan Morgan', 'MID', 91, 115)], 'FWD': [('Sebastian Perry', 'FWD', 95, 150), ('Adrian Collins', 'FWD', 85, 90)], 'total_cost': 727}
Team 2: {'GK': [('Ryan Mitchell', 'GK', 83, 85)], 'DEF': [('Lucas Bennett', 'DEF', 85, 90), ('Owen Parker', 'DEF', 88, 100)], 'MID': [('Connor Hayes', 'MID', 89, 105), ('Gavin Richardson', 'MID', 87, 95)], 'FWD': [('Landon Powell', 'FWD', 89, 110), ('Julian Scott', 'FWD', 92, 130)], 'total_cost': 715}
Team 3: {'GK': [('Blake Henderson', 'GK', 87, 95)], 'DEF': [('Owen Parker

In [ ]:
epa ya nao tou a perceber